# Test run for Hort CA

In [1]:
# Import required modules
import requests
import pandas as pd

In [24]:
# Datas dict to send
datas = {
  "state": "wa_sw",
  "crops": [
    {
      "type": "Annual Hort",
      "averageYield": 405.96,
      "areaSown": 1.3,
      "ureaApplication": 0,
      "nonUreaNitrogen": 698.4,
      "ureaAmmoniumNitrate": 0,
      "phosphorusApplication": 194.96,
      "potassiumApplication": 1042.4,
      "sulfurApplication": 136.92,
      "rainfallAbove600": bool('true'),
      "ureaseInhibitorUsed": bool('true'),
      "nitrificationInhibitorUsed": bool('true'),
      "fractionOfAnnualCropBurnt": 0,
      "herbicideUse": 0,
      "glyphosateOtherHerbicideUse": 0,
      "electricityAllocation": 0.71,
      "limestone": 0,
      "limestoneFraction": 0,
      "dieselUse": 3781,
      "petrolUse": 3955,
      "lpg": 0,
      "refrigerants": [
        {
          "refrigerant": "HFC-134a",
          "chargeSize": 160
        }
      ]
    },
    {
      "type": "Annual Hort",
      "averageYield": 167.75,
      "areaSown": 0.2,
      "ureaApplication": 0,
      "nonUreaNitrogen": 134.05,
      "ureaAmmoniumNitrate": 0,
      "phosphorusApplication": 37.91,
      "potassiumApplication": 202.69,
      "sulfurApplication": 26.62,
      "rainfallAbove600": bool('true'),
      "ureaseInhibitorUsed": bool('true'),
      "nitrificationInhibitorUsed": bool('true'),
      "fractionOfAnnualCropBurnt": 0,
      "herbicideUse": 0,
      "glyphosateOtherHerbicideUse": 0,
      "electricityAllocation": 29,
      "limestone": 0,
      "limestoneFraction": 0,
      "dieselUse": 735,
      "petrolUse": 769,
      "lpg": 0,
      "refrigerants": [
        {
          "refrigerant": "HFC-134a",
          "chargeSize": 160
        }
      ]
    }
  ],
  "electricityRenewable": 30,
  "electricityUse": 9530,
  "vegetation": [
    {
      "vegetation": {
        "region": "South West",
        "treeSpecies": "No tree data available",
        "soil": "No Soil / Tree data available",
        "area": 0,
        "age": 0
      },
      "allocationToCrops": [
        0
      ]
    }
  ]
}

In [4]:
# Set url to Hort tool
url = 'https://emissionscalculator-mtls.production.aiaapi.com/calculator/v1/horticulture'

# Header to send through
headers = {
    "Accept": 'application/json',
    'Content-Type': 'application/json',
    "User-Agent": 'terrawise'
}

# Key and perm
key = 'carbon-calculator-integration.key'
pem = 'aiaghg-terrawise.pem'

In [25]:
# Send the request
rq = requests.post(url, headers=headers, json=datas, cert=(pem, key))

# Check status code
print(rq.status_code)

200


In [29]:
# Manipulated into pd.Dataframe

response_dict = rq.json()

desired_crop = []

for i in range(len(datas['crops'])):
    desired_crop.append(datas['crops'][i]['type'])

# Empty list to store the scope, metric, 
# value of the response dict
metrics_list = []
by_crop = []

for keys, values in response_dict.items():
    if keys != 'intermediate' and keys != 'metaData': # Skip intermediate (crop specific) and metaDate
        # Check if the value of the key is a dictionary
        if isinstance(values, dict):
            for key, value in values.items():
                # Check if the value is not a list
                if not isinstance(value, list):
                    metrics_list.append(
                        {
                            'scope': keys,
                            'metric': key,
                            'value': value
                        }
                    )
                else:
                    for i in range(len(value)):
                        # List type value breaks the result into crop type
                        # Append based on the crop type
                        metrics_list.append(
                            {
                            'scope': keys,
                            'metric': desired_crop[i],
                            'value': value[i]
                            }
                        )
        else:
            # For list type values
            for i in range(len(values)):
                if keys == 'intensitiesWithSequestration': # this key has dictionary type value
                    for key, value in values[i].items():
                        metrics_list.append(
                            {
                                'scope': keys + '_' + desired_crop[i],
                                'metric': key,
                                'value': value
                            }
                        )
                else:
                    metrics_list.append(
                    {
                        'scope': keys,
                        'metric': desired_crop[i],
                        'value': values[i]
                    }
                )
                    
for dictionary in response_dict['intermediate']:
    df = []
    for scope, dict_within in dictionary.items():
        if isinstance(dict_within, dict):
            for key, value in dict_within.items():
                df.append(
                    {
                        'scope': scope,
                        'metric': key,
                        'value': value
                    }
                )
        else:
            df.append(
                {
                    'scope': scope,
                    'metric': "",
                    'value': dict_within
                }
            )
    by_crop.append(df)